In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm
import matplotlib.pyplot as plt
from itertools import combinations
from datetime import datetime

In [2]:
table2_data = pd.read_csv("data/merged_data.csv")
# only need the rows with the data, first 4 rows and last 1 row are not needed
table2_ff5 = pd.read_csv("data/FF5_monthly.csv")
table2_UMD = pd.read_csv("data/FF_Momentum_monthly.csv")
# Rename columns to ensure 'date' column is correctly named

table2_ff5.columns = ['date', 'Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF']
table2_UMD.columns = ['date', 'UMD']

table2_data['date'] = pd.to_datetime(table2_data['date'])
table2_ff5['date'] = pd.to_datetime(table2_ff5['date'], format='%Y%m')
table2_UMD['date'] = pd.to_datetime(table2_UMD['date'], format='%Y%m')

In [3]:
# Time frame for the data
start_date = '1999-07-01'
end_date = '2023-12-31'
table2_data = table2_data[(table2_data['date'] >= start_date) & (table2_data['date'] <= end_date)]

# Get the remaining factors belonging to the time frame
unique_predictors = table2_data['predictor'].unique()
print(f'The list of predictors is: \n{unique_predictors}')
print(f'The number of predictors we selected are: {len(unique_predictors)}')

The list of predictors is: 
['Accruals' 'AnalystValue' 'AssetGrowth' 'BM' 'BPEBM' 'Beta'
 'BetaLiquidityPS' 'BookLeverage' 'CBOperProf' 'CF' 'CPVolSpread'
 'ChAssetTurnover' 'ChNWC' 'CompEquIss' 'CompositeDebtIssuance'
 'Coskewness' 'CustomerMomentum' 'DolVol' 'EBM' 'EP' 'EarningsSurprise'
 'FirmAge' 'Frontier' 'GP' 'Herf' 'High52' 'IdioVol3F' 'Illiquidity'
 'IntMom' 'InvGrowth' 'LRreversal' 'MaxRet' 'Mom12m' 'Mom6m' 'Mom6mJunk'
 'MomOffSeason' 'MomOffSeason06YrPlus' 'MomOffSeason11YrPlus'
 'MomOffSeason16YrPlus' 'MomSeason' 'MomSeason06YrPlus'
 'MomSeason11YrPlus' 'MomSeason16YrPlus' 'MomSeasonShort' 'NOA' 'OperProf'
 'PS' 'RDAbility' 'RIVolSpread' 'ResidualMomentum' 'RoE' 'SP' 'STreversal'
 'ShareIss1Y' 'ShareIss5Y' 'Size' 'VolMkt' 'VolSD' 'XFIN' 'cfp' 'roaq'
 'std_turn']
The number of predictors we selected are: 62


In [4]:
# first get the monthly returns for each stock, sum of the daily returns by month
table2_data['month'] = table2_data['date'].dt.month
table2_data['year'] = table2_data['date'].dt.year
table2_data['month_year'] = table2_data['date'].dt.to_period('M')
# sum the daily returns by month
table2_data_monthly = table2_data.groupby(['month_year', 'predictor'])[['port01','port02','port03','port04','port05','portLS']].sum().reset_index()
table2_data_monthly['date'] = table2_data_monthly['month_year'].dt.to_timestamp()
table2_data_monthly = table2_data_monthly.drop(['month_year'], axis=1)
table2_data_monthly = table2_data_monthly.sort_values(by=['predictor','date'])
table2_data_monthly = table2_data_monthly.reset_index(drop=True)
# put date in first column
cols = table2_data_monthly.columns.tolist()
cols = cols[-1:] + cols[:-1]
table2_data_monthly = table2_data_monthly[cols]
table2_data_monthly.set_index('date', inplace=True)
# divide 100
table2_data_monthly[['port01', 'port02', 'port03', 'port04', 'port05', 'portLS']] = table2_data_monthly[['port01', 'port02', 'port03', 'port04', 'port05', 'portLS']] / 100
table2_data_monthly['portLS2'] = table2_data_monthly['port05'] + table2_data_monthly['port04'] - table2_data_monthly['port02'] - table2_data_monthly['port01']
table2_data_monthly

,predictor,port01,port02,port03,port04,port05,portLS,portLS2
date,,,,,,,,
1999-07-01,Accruals,-0.042332,-0.033650,-0.024488,-0.026960,-0.020499,0.021833,0.028523
1999-08-01,Accruals,-0.041417,-0.026728,-0.004585,-0.000784,0.009333,0.050751,0.076694
1999-09-01,Accruals,-0.007618,-0.058277,-0.029534,-0.020111,-0.003396,0.004222,0.042388
1999-10-01,Accruals,0.054138,0.078025,0.054857,0.054379,0.063649,0.009511,-0.014135
1999-11-01,Accruals,0.020576,0.023920,0.005167,0.038106,0.046697,0.026122,0.040308
...,...,...,...,...,...,...,...,...
2023-08-01,std_turn,-0.188815,-0.068048,-0.025704,-0.032045,-0.022582,0.166233,0.202236
2023-09-01,std_turn,-0.067067,-0.050836,-0.034021,-0.052222,-0.044330,0.022738,0.021352
2023-10-01,std_turn,-0.138222,-0.100143,-0.061419,-0.033710,-0.044634,0.093587,0.160021


## Table 2
### Panel A

In [5]:
# get the data for table2 Panel A
# Pivot the data to have each predictor's portLS as a column
table2_LS1 = table2_data_monthly.pivot(columns='predictor', values='portLS') # long top 20%, short bottom 20%
table2_LS2 = table2_data_monthly.pivot(columns='predictor', values='portLS2') # Long top 40%, short bottom 40%

### $ Double-check!$

In [6]:
# Define Factor Momentum Strategy Function
def factor_momentum_strategy(returns, L, H, n_long_short):

    strategy_returns = pd.Series(index=returns.index, dtype=float)
    N = returns.shape[1]  
    for t in range(L, len(returns)):
        # Formation period: t-L to t-1 (using full data history up to t)
        past_returns = returns.iloc[t-L:t].mean()  # Average return over last L months
        # Rank factors
        ranked_factors = past_returns.sort_values(ascending=False)
        # Select top and bottom n_long_short factors
        long_factors = ranked_factors.index[:n_long_short]
        short_factors = ranked_factors.index[-n_long_short:]
        
        # Holding period: t to t+H-1
        if H == 1:
            # For H=1, take the next month's return
            long_returns = returns.iloc[t][long_factors].mean()
            short_returns = returns.iloc[t][short_factors].mean()
            strategy_returns.iloc[t] = long_returns - short_returns
        else:
            holding_returns = []
            for h in range(H):
                if t - h >= L:  # to ensure the data have enough history L months
                    past_h = returns.iloc[t-h-L:t-h].mean()  # to calculate the average return over last L months
                    ranked_h = past_h.sort_values(ascending=False)
                    long_h = ranked_h.index[:n_long_short]
                    short_h = ranked_h.index[-n_long_short:]

                    if t + h < len(returns):  # ensure the data not out of range
                        future_h = returns.iloc[t+h][long_h].mean() - returns.iloc[t+h][short_h].mean()
                        holding_returns.append(future_h)

            strategy_returns.iloc[t] = np.mean(holding_returns) if holding_returns else np.nan
    
    return strategy_returns

In [7]:
# Step 3: Compute Strategies
N = len(table2_data_monthly['predictor'].unique())  # Fixed number of factors
n = max(round(3/20 * N), 1)  # n = 9

# Long TOP 20% and Short BOTTOM 20% of factors
# L=1, H=1
LS1_1_1 = factor_momentum_strategy(table2_LS1, L=1, H=1, n_long_short=n)
# L=6, H=6
LS1_6_6 = factor_momentum_strategy(table2_LS1, L=6, H=6, n_long_short=n)

# Long TOP 40% and Short BOTTOM 40% of factors
# L=1, H=1
LS2_1_1 = factor_momentum_strategy(table2_LS2, L=1, H=1, n_long_short=n)
# L=6, H=6
LS2_6_6 = factor_momentum_strategy(table2_LS2, L=6, H=6, n_long_short=n)


### $ Double-check!$

In [8]:
# Panel A - Compute Annualized Returns, Std Dev, and t-values
def compute_statistics(returns, H):
    returns = returns.dropna()
    # Annualized return: Monthly mean * 12
    ann_return = returns.mean() * 12 * 100  # In percentage
    # Annualized standard deviation: Monthly std * sqrt(12)
    ann_std = returns.std() * np.sqrt(12) 
    # t-value
    t_value = returns.mean() / ( (returns.std() / np.sqrt(len(returns))) )
    return ann_return, ann_std, t_value

In [9]:

def filter_returns(returns, start_date, end_date):
    return returns[(returns.index >= start_date) & (returns.index <= end_date)].dropna()

results = []
strategies = {
    'LS1_1_1': {'L': 1, 'H': 1, 'Factor': 'LS1'},
    'LS1_6_6': {'L': 6, 'H': 6, 'Factor': 'LS1'},
    'LS2_1_1': {'L': 1, 'H': 1, 'Factor': 'LS2'},
    'LS2_6_6': {'L': 6, 'H': 6, 'Factor': 'LS2'}
}

time_periods = [
    ('all', datetime(2000, 1, 1), datetime(2023, 12, 31)),
    ('sample', datetime(2000, 1, 1), datetime(2016, 12, 31)),
    ('post', datetime(2017, 1, 1), datetime(2023, 12, 31))
]

for period_name, start_date, end_date in time_periods:
    for strategy_name, params in strategies.items():
        L, H, factor = params['L'], params['H'], params['Factor']
        filtered_returns = filter_returns(globals()[strategy_name], start_date, end_date)
        ann_return, ann_std, t_value = compute_statistics(filtered_returns, H)
        results.append({
            'Factor': factor,
            'Formation period': L,
            'Holding period': H,
            'Period': period_name,
            'Annualized return': ann_return,
            'Standard deviation': ann_std,
            't-value': t_value
        })


table2_pa_results = pd.DataFrame(results)
table2_pa_results['Period'] = pd.Categorical(table2_pa_results['Period'], categories=['all', 'sample', 'post'], ordered=True)
table2_pa_results = table2_pa_results.sort_values(by=['Period', 'Factor', 'Formation period', 'Holding period'])
table2_pa_results = table2_pa_results[['Period', 'Factor', 'Formation period', 'Holding period', 'Annualized return', 'Standard deviation', 't-value']]
table2_pa_results = table2_pa_results.round(2) 

print("Panel A: Annualized percent returns and standard deviations")
table2_pa_results

Panel A: Annualized percent returns and standard deviations


,Period,Factor,Formation period,Holding period,Annualized return,Standard deviation,t-value
0,all,LS1,1,1,5.64,0.25,1.09
1,all,LS1,6,6,0.09,0.09,0.05
2,all,LS2,1,1,12.12,0.39,1.54
3,all,LS2,6,6,1.37,0.14,0.46
4,sample,LS1,1,1,5.25,0.28,0.79
5,sample,LS1,6,6,0.12,0.10,0.05
6,sample,LS2,1,1,11.46,0.42,1.14
7,sample,LS2,6,6,0.81,0.16,0.21
8,post,LS1,1,1,6.61,0.20,0.89
9,post,LS1,6,6,0.03,0.07,0.01


### Panel B

In [10]:
start_date = '2000-01-01'
end_date = '2023-12-31'
# combine the FF5 and UMD data
table2_ff5 = table2_ff5[(table2_ff5['date'] >= start_date) & (table2_ff5['date'] <= end_date)]
table2_UMD = table2_UMD[(table2_UMD['date'] >= start_date) & (table2_UMD['date'] <= end_date)]
table2_FF5_UMD = pd.merge(table2_ff5, table2_UMD, on='date', how='inner')
table2_FF5_UMD.set_index('date', inplace=True)
table2_FF5_UMD = table2_FF5_UMD[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF','UMD']] / 100
table2_FF5_UMD
print(table2_FF5_UMD)

            Mkt-RF     SMB     HML     RMW     CMA      RF     UMD
date                                                              
2000-01-01 -0.0474  0.0442 -0.0188 -0.0629  0.0472  0.0041  0.0192
2000-02-01  0.0245  0.1828 -0.0959 -0.1865 -0.0048  0.0043  0.1820
2000-03-01  0.0520 -0.1532  0.0813  0.1179 -0.0159  0.0047 -0.0683
2000-04-01 -0.0640 -0.0501  0.0726  0.0766  0.0565  0.0046 -0.0839
2000-05-01 -0.0442 -0.0381  0.0475  0.0413  0.0137  0.0050 -0.0898
...            ...     ...     ...     ...     ...     ...     ...
2023-08-01 -0.0239 -0.0368 -0.0108  0.0342 -0.0237  0.0045  0.0377
2023-09-01 -0.0524 -0.0179  0.0145  0.0185 -0.0084  0.0043  0.0024
2023-10-01 -0.0318 -0.0405  0.0019  0.0247 -0.0067  0.0047  0.0168
2023-11-01  0.0883 -0.0011  0.0166 -0.0381 -0.0099  0.0044  0.0276
2023-12-01  0.0487  0.0733  0.0492 -0.0304  0.0130  0.0043 -0.0548

[288 rows x 7 columns]


In [11]:
# Panel B
# combine the LS data
table2_LS = pd.concat([LS1_1_1, LS1_6_6, LS2_1_1, LS2_6_6], axis=1)
table2_LS.columns = ['LS1_1_1', 'LS1_6_6', 'LS2_1_1', 'LS2_6_6']
table2_LS.dropna(inplace=True)
print(table2_LS)

             LS1_1_1   LS1_6_6   LS2_1_1   LS2_6_6
date                                              
2000-01-01  0.001087  0.025835  0.047680  0.050837
2000-02-01  0.201613  0.080248  0.259084  0.105430
2000-03-01 -0.243030 -0.193091 -0.320838 -0.276350
2000-04-01  0.115142 -0.067419  0.220804 -0.092238
2000-05-01  0.177986  0.006661  0.309098  0.013015
...              ...       ...       ...       ...
2023-08-01 -0.127709 -0.008088 -0.144828 -0.017041
2023-09-01  0.055721  0.008494  0.046506  0.009720
2023-10-01  0.037921  0.013672  0.058097  0.020384
2023-11-01 -0.036180 -0.026301 -0.072467 -0.029742
2023-12-01  0.046741 -0.091450  0.074537 -0.072676

[288 rows x 4 columns]


In [12]:
# Define the dependent variables (strategy returns from Panel A)
dependent_vars = {
   'LS1_1_1': table2_LS['LS1_1_1'],  # Standard factor momentum, L=1, H=1
   'LS1_6_6': table2_LS['LS1_6_6'],  # Standard factor momentum, L=6, H=6
   'LS2_1_1': table2_LS['LS2_1_1'],  # Industry-adjusted factor momentum, L=1, H=1
   'LS2_6_6': table2_LS['LS2_6_6']   # Industry-adjusted factor momentum, L=6, H=6
}

# Define the independent variables (factors, excluding RF as per factor model convention)
factors = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'UMD']

# Function to run spanning regression
def run_spanning_regression(dep_var, ind_vars, label):
    X = sm.add_constant(ind_vars)
    model = sm.OLS(dep_var, X).fit()  
    print(f"\nRegression Results for {label}:")
    print(model.summary())
    return model.params['const'], model.tvalues['const']

In [13]:
# Run regressions for each strategy ( FF5 + UMD only)
results = {}

# 1. Long-Short 20% (L=1, H=1)
ind_vars_1 = table2_FF5_UMD[factors]
alpha_1, t_stat_1 = run_spanning_regression(dependent_vars['LS1_1_1'], ind_vars_1, 'Long-Short 20%, L=1, H=1')
results['LS1_1_1'] = (alpha_1, t_stat_1)  # Convert to percent


Regression Results for Long-Short 20%, L=1, H=1:
                            OLS Regression Results                            
Dep. Variable:                LS1_1_1   R-squared:                       0.147
Model:                            OLS   Adj. R-squared:                  0.129
Method:                 Least Squares   F-statistic:                     8.066
Date:                Wed, 12 Mar 2025   Prob (F-statistic):           4.76e-08
Time:                        20:03:10   Log-Likelihood:                 366.77
No. Observations:                 288   AIC:                            -719.5
Df Residuals:                     281   BIC:                            -693.9
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
co

In [14]:
# 2. Long-Short 20% (L=6, H=6)
ind_vars_2 = table2_FF5_UMD[factors]
alpha_2, t_stat_2 = run_spanning_regression(dependent_vars['LS1_6_6'], ind_vars_2, 'Long-Short 20%, L=6, H=6')
results['LS1_6_6'] = (alpha_2, t_stat_2)


Regression Results for Long-Short 20%, L=6, H=6:
                            OLS Regression Results                            
Dep. Variable:                LS1_6_6   R-squared:                       0.117
Model:                            OLS   Adj. R-squared:                  0.098
Method:                 Least Squares   F-statistic:                     6.187
Date:                Wed, 12 Mar 2025   Prob (F-statistic):           4.10e-06
Time:                        20:03:10   Log-Likelihood:                 654.15
No. Observations:                 288   AIC:                            -1294.
Df Residuals:                     281   BIC:                            -1269.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
co

In [15]:
# 3. Long-Short 40% (L=1, H=1)
ind_vars_3 = table2_FF5_UMD[factors]
alpha_3, t_stat_3 = run_spanning_regression(dependent_vars['LS2_1_1'], ind_vars_3, 'Long-Short 40%, L=1, H=1')
results['LS2_1_1'] = (alpha_3, t_stat_3)


Regression Results for Long-Short 40%, L=1, H=1:
                            OLS Regression Results                            
Dep. Variable:                LS2_1_1   R-squared:                       0.166
Model:                            OLS   Adj. R-squared:                  0.148
Method:                 Least Squares   F-statistic:                     9.303
Date:                Wed, 12 Mar 2025   Prob (F-statistic):           2.62e-09
Time:                        20:03:10   Log-Likelihood:                 249.22
No. Observations:                 288   AIC:                            -484.4
Df Residuals:                     281   BIC:                            -458.8
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
co

In [16]:
# 4. Long-Short 40% (L=6, H=6)
ind_vars_4 = table2_FF5_UMD[factors]
alpha_4, t_stat_4 = run_spanning_regression(dependent_vars['LS2_6_6'], ind_vars_4, 'Long-Short 40%, L=6, H=6')
results['LS2_6_6'] = (alpha_4, t_stat_4)


Regression Results for Long-Short 40%, L=6, H=6:
                            OLS Regression Results                            
Dep. Variable:                LS2_6_6   R-squared:                       0.102
Model:                            OLS   Adj. R-squared:                  0.083
Method:                 Least Squares   F-statistic:                     5.337
Date:                Wed, 12 Mar 2025   Prob (F-statistic):           3.10e-05
Time:                        20:03:10   Log-Likelihood:                 522.91
No. Observations:                 288   AIC:                            -1032.
Df Residuals:                     281   BIC:                            -1006.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
co